# load taunus-medusa data and use baseline filter to get estimates for summer 2025

In [ ]:
import xarray as xr
from pathlib import Path

# %matplotlib widget
from matplotlib import pyplot as plt
import seaborn as sns

data_dir = Path("../data/raw/")
fname_ch2cl2 = data_dir / "AGAGE-GCMS-Medusa_TOB_ch2cl2.nc"
fname_ch3br = data_dir / "AGAGE-GCMS-Medusa_TOB_ch3br.nc"
fname_ch3cl = data_dir / "AGAGE-GCMS-Medusa_TOB_ch3cl.nc"
fname_pce = data_dir / "AGAGE-GCMS-Medusa_TOB_pce.nc"

# CH2Cl2
This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


![image.png](../external_plots/Screenshot%202026-02-02%20134736.png)

In [ ]:
ch2cl2 = xr.open_dataset(fname_ch2cl2)

In [ ]:
ch2cl2

In [ ]:
fig, ax = plt.subplots()
ax.scatter(ch2cl2.time, ch2cl2.mf)
plt.show()

In [ ]:
import IAU_baseline.baseline as base
import pandas as pd

In [ ]:
df = ch2cl2.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from vsls_2025.plotting import run_interactive
baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)
baseline = ["pollution event" if x > 0 else "baseline" for x in baseline[0]]
df["Classification"] = baseline

In [ ]:
df = ch2cl2.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

baseline = base.find_base(func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)
baseline = ["pollution event" if x > 0 else "baseline" for x in baseline[0]]
df["Classification"] = baseline
plt.show()

In [ ]:
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

In [ ]:
df

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["datetime"], df["mf"])
plt.show()